Com o novo sistema do ICB, temos agora os arquivos .ics disponibilizados pelo instituto. Yay, sucesso.

Para raspar isso de forma simples, vamos usar o Selenium para rodar o Python headless

In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Firefox()
driver.get("https://ww3.icb.usp.br/calendario-de-eventos/mes/")



In [36]:
event_month = driver.find_elements_by_xpath('//*[@id="tribe-bar-date"]')[0]
event_month.clear()
event_month.send_keys("04/2020")
event_button = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div/div[2]/div[2]/form/div/div/div/div[3]/input')[0]
event_button.click()
download_button = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div/div[2]/div[3]/a')[0]
download_button.click()

Hm, o site tem um bug que só baixa os eventos de janeiro. Fiz o report em 30/01/2020. Veremos.  Eles irão trabalhar no bug. Enquanto isso, vou tentar outra abordagem: pegar todos os links do calendário e raspar as páginas. 


In [155]:
from bs4 import BeautifulSoup
import requests
import re


In [156]:
html_page = requests.get("https://ww3.icb.usp.br/calendario-de-eventos/mes/")
get_all_links(html_page)

https://ww3.icb.usp.br/eventos/ix-curso-de-verao-em-biologia-celular-e-do-desenvolvimento/
https://ww3.icb.usp.br/eventos/neuromodulation-as-a-novel-approach-for-regulation-of-immune-and-metabolic-dysfunction/
https://ww3.icb.usp.br/eventos/comprehensive-characterization-of-immune-cells-within-the-tumor-microenvironment/
https://ww3.icb.usp.br/eventos/coronavirus-2019-mitos-e-realidade/
https://ww3.icb.usp.br/eventos/seminarios-especiais-em-imunologia/
https://ww3.icb.usp.br/eventos/prevencao-primaria-e-secundaria-para-cancer/


In [179]:


%run "/home/lubianat/Documents/Projects/side/seminUSP/src/cal_extract.py"

In [180]:



def get_event_ical(event_url):
    event_page = requests.get(event_url)
    event_soup = BeautifulSoup(event_page.text)

    event_title = event_soup.find('h1',attrs={"class":"tribe-events-single-event-title"}, text = True)
    event_title = event_title.get_text()
    print(event_title)
    infos = event_soup.find(class_="tribe-events-single-event-description tribe-events-content")
    info_text = infos.get_text()
    magic_event = magic_extract_event(info_text)
    magic_event.add('summary', event_title )
    return(magic_event)




In [181]:
from icalendar import Calendar, Event
import datetime as dt


cal = Calendar()

soup = BeautifulSoup(html_page.text)
tags =  soup.find_all('a')
for tag in tags:
    link = tag.get('href')
    if '/eventos/' in link:
            try:
                event_ical = get_event_ical(link)
                cal.add_component(event_ical)
            except Exception as ex:
                print(ex)

                import tempfile, os


IX Curso de Verão em Biologia Celular e do Desenvolvimento
[]
invalid literal for int() with base 10: 'a '
Neuromodulation as a novel approach for regulation of immune and metabolic dysfunction
[]
list index out of range
Comprehensive characterization of immune cells within the tumor microenvironment
[]
12
Coronavírus 2019: Mitos e Realidade
Prof. Sebastião Timo Iaria
12
Seminários Especiais em Imunologia
[]
14
Prevenção Primária e Secundária para Câncer
[]
12
local variable 'local_final' referenced before assignment


In [182]:
directory = tempfile.mkdtemp()
f = open(os.path.join(directory, 'example.ics'), 'wb')
f.write(cal.to_ical())
f.close()